In [1]:
# Tải các thư viện cần thiết
import os
import sys
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import time

# Thêm đường dẫn src vào hệ thống
sys.path.append(os.path.abspath('../src'))

# Import các module từ src - GIỐNG HỆT FILE GỐC
from preprocessing import preprocess_pipeline
from scattering_features import create_scattering_transform, extract_and_scale_scattering_features
from dataset_loader import find_data_files, get_patient_id_from_path, get_murmur_label, split_and_save_dataset

# CHỈ KHÁC Ở ĐÂY: Import segmentation mới (Hilbert-based) thay vì segmentation_pipeline
from segmentation import segment_cardiac_cycles

# Cài đặt cho plots
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
print("Các thư viện và module đã được tải thành công!")

Các thư viện và module đã được tải thành công!


In [2]:
# 2. Định nghĩa các đường dẫn và tham số
# Đường dẫn đến thư mục dữ liệu
# LƯU Ý: Bạn cần đảm bảo đã tải và giải nén bộ dữ liệu PhysioNet vào thư mục này.
# Cấu trúc dự kiến: ../data/raw/training_data/...
RAW_DATA_DIR = '../data/raw/training_data/' 
PROCESSED_DATA_DIR = '../data/processed/'

# Giả định file metadata có tên là 'training_data.csv' và nằm trong RAW_DATA_DIR
# Đây là file chứa thông tin bệnh nhân và nhãn 'Murmur'
METADATA_FILE = os.path.join(os.path.dirname(RAW_DATA_DIR.rstrip('/')), 'training_data.csv')

# Tham số xử lý - GIỐNG HỆT FILE GỐC
TARGET_SR = 2000
TARGET_CYCLE_LEN = 1600

# Tham số Scattering Transform
J_SCATTER = 2
Q_SCATTER = 8

# Tạo thư mục lưu dữ liệu đã xử lý nếu chưa tồn tại
if not os.path.exists(PROCESSED_DATA_DIR):
    os.makedirs(PROCESSED_DATA_DIR)

print(f"Thư mục dữ liệu thô: {os.path.abspath(RAW_DATA_DIR)}")
print(f"Thư mục lưu dữ liệu xử lý: {os.path.abspath(PROCESSED_DATA_DIR)}")
print(f"File metadata: {os.path.abspath(METADATA_FILE)}")

Thư mục dữ liệu thô: C:\Users\Nguyen duc phat\OneDrive\Máy tính\Heart-Project\data\raw\training_data
Thư mục lưu dữ liệu xử lý: C:\Users\Nguyen duc phat\OneDrive\Máy tính\Heart-Project\data\processed
File metadata: C:\Users\Nguyen duc phat\OneDrive\Máy tính\Heart-Project\data\raw\training_data.csv


In [3]:
# 3. Tải metadata và tìm file dữ liệu
print("Đang tải metadata...")
if os.path.exists(METADATA_FILE):
    metadata_df = pd.read_csv(METADATA_FILE)
    print(f"Đã tải {len(metadata_df)} bản ghi từ file metadata.")
    print(f"Các cột có sẵn: {list(metadata_df.columns)}")
else:
    print(f"Không tìm thấy file metadata: {METADATA_FILE}")
    metadata_df = None

# Tìm các cặp file WAV và TSV
print("\nĐang tìm các file dữ liệu...")
file_pairs = find_data_files(RAW_DATA_DIR)
if file_pairs:
    wav_files, tsv_files = file_pairs
    print(f"Tìm thấy {len(wav_files)} cặp file WAV/TSV để xử lý.")
else:
    print("Không tìm thấy file dữ liệu nào!")
    wav_files, tsv_files = [], []

Đang tải metadata...
Đã tải 942 bản ghi từ file metadata.
Các cột có sẵn: ['Patient ID', 'Recording locations:', 'Age', 'Sex', 'Height', 'Weight', 'Pregnancy status', 'Murmur', 'Murmur locations', 'Most audible location', 'Systolic murmur timing', 'Systolic murmur shape', 'Systolic murmur grading', 'Systolic murmur pitch', 'Systolic murmur quality', 'Diastolic murmur timing', 'Diastolic murmur shape', 'Diastolic murmur grading', 'Diastolic murmur pitch', 'Diastolic murmur quality', 'Outcome', 'Campaign', 'Additional ID']

Đang tìm các file dữ liệu...
Tìm thấy 3163 cặp file WAV/TSV để xử lý.


In [4]:
# ===== GIAI ĐOẠN 1: THU THẬP CHU KỲ THÔ =====
import librosa
import random

# Danh sách lưu tất cả chu kỳ thô và thông tin liên quan
all_raw_cycles_info = []
initial_durations_flat = []
error_count = 0
processed_files_count = 0

print(f"Bắt đầu xử lý {len(wav_files)} file WAV để thu thập chu kỳ tim...")

# Thu thập chu kỳ từ tất cả các file
for wav_path, tsv_path in tqdm(zip(wav_files, tsv_files), total=len(wav_files), desc="Đang xử lý các file"):
    try:
        # 1. Tải tín hiệu âm thanh
        signal, sr = librosa.load(wav_path, sr=TARGET_SR)
        
        # 2. Tiền xử lý tín hiệu
        signal_processed = preprocess_pipeline(signal, sr)
        
        # 3. Lấy patient ID và nhãn
        patient_id = get_patient_id_from_path(wav_path)
        murmur_label = get_murmur_label(patient_id, metadata_df) if metadata_df is not None else None
        
        if murmur_label is None:
            continue
            
        # 4. KHÁC BIỆT: Sử dụng Hilbert Transform thay vì segmentation_pipeline
        cardiac_cycles = segment_cardiac_cycles(signal_processed, sr)
        
        if cardiac_cycles:
            # Thu thập tất cả chu kỳ từ file này
            for cycle in cardiac_cycles:
                all_raw_cycles_info.append({
                    'cycle': cycle,
                    'patient_id': patient_id,
                    'label': murmur_label,
                    'file_path': wav_path
                })
                initial_durations_flat.append(len(cycle))
            
        processed_files_count += 1
            
    except Exception as e:
        print(f"Lỗi khi xử lý {os.path.basename(wav_path)}: {e}")
        error_count += 1

print(f"\nHoàn tất thu thập chu kỳ thô:")
print(f"- Đã xử lý thành công: {processed_files_count} file")
print(f"- Lỗi: {error_count} file")
print(f"- Tổng số chu kỳ thu thập được: {len(all_raw_cycles_info)}")
if initial_durations_flat:
    print(f"- Độ dài chu kỳ trung bình: {np.mean(initial_durations_flat):.2f} samples")
    print(f"- Phạm vi độ dài: {min(initial_durations_flat)} - {max(initial_durations_flat)} samples")

Bắt đầu xử lý 3163 file WAV để thu thập chu kỳ tim...


Đang xử lý các file:   0%|          | 0/3163 [00:00<?, ?it/s]

C:\Users\Nguyen duc phat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\intervals.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


Bắt đầu phân đoạn chu kỳ tim bằng Hilbert Transform cho tín hiệu 20576 samples
Đã tính Hilbert envelope cho 20576 samples
Đã làm mượt envelope với cửa sổ 161 samples
Ngưỡng prominence thích ứng được tính: 1.8391
Tìm thấy 20 peaks.
Vị trí S1 (samples): [  623  1764  2905  4064  5220  6332  7502  8668  9820 10957]...
Đã tách được 19 chu kỳ tim bằng Hilbert Transform
Bắt đầu phân đoạn chu kỳ tim bằng Hilbert Transform cho tín hiệu 57568 samples
Đã tính Hilbert envelope cho 57568 samples
Đã làm mượt envelope với cửa sổ 161 samples
Ngưỡng prominence thích ứng được tính: 2.0045
Tìm thấy 27 peaks.
Vị trí S1 (samples): [ 1259  4723  8357 10442 12770 13929 15092 16248 19706 22037]...
Đã tách được 26 chu kỳ tim bằng Hilbert Transform
Bắt đầu phân đoạn chu kỳ tim bằng Hilbert Transform cho tín hiệu 18784 samples
Đã tính Hilbert envelope cho 18784 samples
Đã làm mượt envelope với cửa sổ 161 samples
Ngưỡng prominence thích ứng được tính: 1.8822
Tìm thấy 17 peaks.
Vị trí S1 (samples): [  798  1928  

In [ ]:
# ===== GIAI ĐOẠN 2: LỌC CHU KỲ THEO PHÂN VỊ P10–P90 =====
import numpy as np

# Tính phân vị P10 và P90 của độ dài chu kỳ
p10 = np.percentile(initial_durations_flat, 10)
p90 = np.percentile(initial_durations_flat, 90)

print(f"\nLọc chu kỳ theo độ dài từ P10 ({p10:.2f} samples) đến P90 ({p90:.2f} samples)")

# Lọc và lưu kết quả
filtered_cycles_flat = []
filtered_durations_flat = []
all_labels = []
all_patient_ids = []

for item in all_raw_cycles_info:
    cycle = item['cycle']
    duration = len(cycle)
    
    if p10 <= duration <= p90:
        filtered_cycles_flat.append(cycle)
        filtered_durations_flat.append(duration)
        all_labels.append(item['label'])
        all_patient_ids.append(item['patient_id'])

In [6]:
# In thống kê xử lý
print("--- Thống kê quá trình xử lý chu kỳ tim ---")
print(f"Tổng số file .wav đã đọc: {len(file_pairs)}")
print(f"Tổng số chu kỳ tim trước khi lọc: {len(initial_durations_flat)}")
print(f"Tổng số chu kỳ tim sau khi lọc và chuẩn hóa: {len(filtered_durations_flat)}")

if initial_durations_flat:
    print(f"Tỷ lệ chu kỳ được giữ lại: {len(filtered_durations_flat) / len(initial_durations_flat) if len(initial_durations_flat) > 0 else 0:.2%}")

if filtered_durations_flat:
    print(f"Độ dài trung bình của chu kỳ (trước lọc): {np.mean(initial_durations_flat):.2f} samples")
    print(f"Độ dài trung bình của chu kỳ (sau lọc): {np.mean(filtered_durations_flat):.2f} samples")
    
    # Thống kê phân bố theo lớp Present/Absent
    print("\n--- Phân bố chu kỳ theo lớp ---")
    unique_labels, counts = np.unique(all_labels, return_counts=True)
    for label, count in zip(unique_labels, counts):
        percentage = (count / len(all_labels)) * 100
        label_name = "Present" if label == 1 else "Absent"
        print(f"Lớp {label_name}: {count:,} chu kỳ ({percentage:.2f}%)")
    
    # Thống kê chi tiết hơn
    present_count = sum(1 for label in all_labels if label == 1)
    absent_count = sum(1 for label in all_labels if label == 0)
    total_cycles = len(all_labels)
    
    print(f"\nTổng kết:")
    print(f"- Chu kỳ có Murmur (Present): {present_count:,} ({present_count/total_cycles*100:.2f}%)")
    print(f"- Chu kỳ không có Murmur (Absent): {absent_count:,} ({absent_count/total_cycles*100:.2f}%)")
    print(f"- Tỷ lệ Present/Absent: 1:{absent_count/present_count:.2f}")
    
else:
    print("Không có chu kỳ nào được xử lý để tính toán độ dài.")

--- Thống kê quá trình xử lý chu kỳ tim ---
Tổng số file .wav đã đọc: 2
Tổng số chu kỳ tim trước khi lọc: 96877
Tổng số chu kỳ tim sau khi lọc và chuẩn hóa: 77637
Tỷ lệ chu kỳ được giữ lại: 80.14%
Độ dài trung bình của chu kỳ (trước lọc): 1360.18 samples
Độ dài trung bình của chu kỳ (sau lọc): 940.45 samples

--- Phân bố chu kỳ theo lớp ---
Lớp Absent: 60,994 chu kỳ (78.56%)
Lớp Present: 16,643 chu kỳ (21.44%)

Tổng kết:
- Chu kỳ có Murmur (Present): 16,643 (21.44%)
- Chu kỳ không có Murmur (Absent): 60,994 (78.56%)
- Tỷ lệ Present/Absent: 1:3.66


In [7]:
def pad_or_crop(cycle, target_len=1600):
    if len(cycle) > target_len:
        # Nếu chu kỳ quá dài → cắt ở giữa
        start = (len(cycle) - target_len) // 2
        return cycle[start:start + target_len]
    
    elif len(cycle) < target_len:
        # Nếu chu kỳ quá ngắn → đệm thêm 0 ở cuối
        pad_width = target_len - len(cycle)
        return np.pad(cycle, (0, pad_width), mode='constant')
    
    # Nếu vừa đủ thì không làm gì
    return cycle

# Chuẩn hóa tất cả chu kỳ về cùng độ dài
X_cycles = np.array([pad_or_crop(cycle) for cycle in filtered_cycles_flat])

In [8]:
# 4.5. Cân bằng dữ liệu bằng Undersampling
# Giảm lớp Absent xuống 10k để cân bằng với Present

print("\n--- Cân bằng dữ liệu bằng Undersampling ---")
print(f"Trước cân bằng: Present={len([l for l in all_labels if l == 1])}, Absent={len([l for l in all_labels if l == 0])}")

# Tách dữ liệu theo lớp
present_indices = [i for i, label in enumerate(all_labels) if label == 1]
absent_indices = [i for i, label in enumerate(all_labels) if label == 0]

# Random sampling để lấy 10k mẫu Absent
import random
random.seed(42)  # Để kết quả có thể tái tạo
sampled_absent_indices = random.sample(absent_indices, 16000)

# Kết hợp indices
balanced_indices = present_indices + sampled_absent_indices
random.shuffle(balanced_indices)  # Trộn lại

# Tạo dữ liệu cân bằng
X_cycles_balanced = X_cycles[balanced_indices]
all_labels_balanced = [all_labels[i] for i in balanced_indices]
all_patient_ids_balanced = [all_patient_ids[i] for i in balanced_indices]

print(f"Sau cân bằng: Present={len([l for l in all_labels_balanced if l == 1])}, Absent={len([l for l in all_labels_balanced if l == 0])}")
print(f"Tổng số mẫu sau cân bằng: {len(all_labels_balanced)}")
print(f"Tỷ lệ Present/Absent mới: 1:{len([l for l in all_labels_balanced if l == 0])/len([l for l in all_labels_balanced if l == 1]):.2f}")

# Cập nhật biến để sử dụng cho các bước tiếp theo
X_cycles = X_cycles_balanced
y_labels = np.array(all_labels_balanced)
patient_ids_array = np.array(all_patient_ids_balanced)


--- Cân bằng dữ liệu bằng Undersampling ---
Trước cân bằng: Present=16643, Absent=60994
Sau cân bằng: Present=16643, Absent=16000
Tổng số mẫu sau cân bằng: 32643
Tỷ lệ Present/Absent mới: 1:0.96


In [9]:
# 5. Trích xuất Đặc trưng Scattering
# Khởi tạo Scattering Transform
if X_cycles.shape[0] > 0:
    print(f"Bắt đầu trích xuất đặc trưng scattering cho {X_cycles.shape[0]} chu kỳ...")
    scattering_transform = create_scattering_transform(J=J_SCATTER, Q=Q_SCATTER, shape=TARGET_CYCLE_LEN)

    # Trích xuất và chuẩn hóa đặc trưng
    start_scatter_time = time.time()
    X_scattering, feature_scaler = extract_and_scale_scattering_features(X_cycles, scattering_transform)
    end_scatter_time = time.time()

    print(f"Hoàn thành trích xuất đặc trưng trong {end_scatter_time - start_scatter_time:.2f} giây.")
    print(f"Kích thước của ma trận đặc trưng: {X_scattering.shape}")
else:
    print("Không có dữ liệu chu kỳ để trích xuất đặc trưng.")
    X_scattering = np.array([])

Bắt đầu trích xuất đặc trưng scattering cho 32643 chu kỳ...
Hoàn thành trích xuất đặc trưng trong 127.15 giây.
Kích thước của ma trận đặc trưng: (32643, 3200)


In [10]:
# 6. Chia dữ liệu thành các tập train, validation, và test
if X_scattering.shape[0] > 0:
    print("Bắt đầu chia dữ liệu cân bằng theo bệnh nhân (70/15/15)...")
    (X_train, y_train), (X_val, y_val), (X_test, y_test) = split_and_save_dataset(
        X=X_scattering,
        y=y_labels,
        patient_ids=patient_ids_array,
        output_dir=PROCESSED_DATA_DIR,
        test_size=0.15,
        val_size=0.15
    )

    # In thống kê kích thước các tập
    print("\n--- Thống kê kích thước các tập dữ liệu cân bằng ---")
    print(f"Tập Train: {X_train.shape[0]} mẫu ({y_train.sum()} Murmur, {len(y_train) - y_train.sum()} Normal)")
    print(f"Tập Validation: {X_val.shape[0]} mẫu ({y_val.sum()} Murmur, {len(y_val) - y_val.sum()} Normal)")
    print(f"Tập Test: {X_test.shape[0]} mẫu ({y_test.sum()} Murmur, {len(y_test) - y_test.sum()} Normal)")
    
    # Tính tỷ lệ cân bằng trong từng tập
    print(f"\nTỷ lệ Present/Absent trong từng tập:")
    print(f"- Train: 1:{(len(y_train) - y_train.sum())/y_train.sum():.2f}")
    print(f"- Validation: 1:{(len(y_val) - y_val.sum())/y_val.sum():.2f}")
    print(f"- Test: 1:{(len(y_test) - y_test.sum())/y_test.sum():.2f}")
    print("--------------------------------------------")
else:
    print("Không có dữ liệu để chia.")

Bắt đầu chia dữ liệu cân bằng theo bệnh nhân (70/15/15)...
Đã lưu các tập dữ liệu vào ../data/processed/

--- Thống kê kích thước các tập dữ liệu cân bằng ---
Tập Train: 23184 mẫu (11885 Murmur, 11299 Normal)
Tập Validation: 4679 mẫu (2191 Murmur, 2488 Normal)
Tập Test: 4780 mẫu (2567 Murmur, 2213 Normal)

Tỷ lệ Present/Absent trong từng tập:
- Train: 1:0.95
- Validation: 1:1.14
- Test: 1:0.86
--------------------------------------------


In [11]:
# Lưu toàn bộ đặc trưng và nhãn cân bằng để có thể sử dụng lại nếu cần
if X_scattering.shape[0] > 0:
    full_features_path = os.path.join(PROCESSED_DATA_DIR, 'X_scattering_balanced.npz')
    full_labels_path = os.path.join(PROCESSED_DATA_DIR, 'y_labels_balanced.npy')

    np.savez_compressed(full_features_path, features=X_scattering)
    np.save(full_labels_path, y_labels)

    print(f"Đã lưu toàn bộ {X_scattering.shape[0]} đặc trưng cân bằng vào: {full_features_path}")
    print(f"Đã lưu toàn bộ {y_labels.shape[0]} nhãn cân bằng vào: {full_labels_path}")
    
    # Thống kê cuối cùng
    present_final = sum(y_labels)
    absent_final = len(y_labels) - present_final
    print(f"\n--- Thống kê cuối cùng sau cân bằng ---")
    print(f"Present: {present_final:,} ({present_final/len(y_labels)*100:.1f}%)")
    print(f"Absent: {absent_final:,} ({absent_final/len(y_labels)*100:.1f}%)")
    print(f"Tỷ lệ Present/Absent: 1:{absent_final/present_final:.2f}")
else:
    print("Không có dữ liệu để lưu.")

Đã lưu toàn bộ 32643 đặc trưng cân bằng vào: ../data/processed/X_scattering_balanced.npz
Đã lưu toàn bộ 32643 nhãn cân bằng vào: ../data/processed/y_labels_balanced.npy

--- Thống kê cuối cùng sau cân bằng ---
Present: 16,643 (51.0%)
Absent: 16,000 (49.0%)
Tỷ lệ Present/Absent: 1:0.96
